In [137]:
engine.rollback()

In [1]:
import psycopg2
engine = psycopg2.connect(
    database="betterreads",
    user="betterreadsadmin",
    password="betterreadsadmin",
    host="betterreads-datascience-database.cvmplnwee5ws.us-east-1.rds.amazonaws.com",
    port='5432')

In [2]:
import traceback
error_log = []
traces = []

def q(query, verbosity = False):
    data = 'No Data'
    curs = engine.cursor()
    try:
        curs.execute(query)
        if(verbosity):
            data = curs.fetchall()
    except psycopg2.errors.InvalidTextRepresentation as err:
        start = int(err.diag.statement_position)
        end = query.find('\'')
        replacement = '{'+query[start:end]+'}'
        new_query = query[:start] + replacement + query[start+len(replacement):]
        engine.rollback()
        q(new_query, verbosity)
        
    except Exception as e:
        engine.rollback()
        print(e)
        traceback.print_exc()
        error_log.append(e)

    engine.commit()
    curs.close()
    if(verbosity):
        return data

In [3]:
q10 = 'DROP TABLE "authors";'
q(q10)
q11 = 'DROP TABLE "editions";'
q(q11)
q01 = 'DROP TABLE "works";'
q(q01)

In [15]:
qJoinTable = """
    CREATE TABLE jointable ( "Author_id" INTEGER,
                             "Work_id" INTEGER )
"""

q(qJoinTable)

In [4]:
qAuthors = """
    CREATE TABLE authors ( "Name" TEXT NOT NULL,
                          "Works" TEXT [] NOT NULL,
                          "Key" TEXT NOT NULL,
                          "AuthorID" SERIAL,
                          PRIMARY KEY("AuthorID") );
"""

q(qAuthors)

In [5]:
qEditions = """
    CREATE TABLE editions ( "EditionTitle" TEXT, 
                            "Authors" TEXT [],
                            "Genre" TEXT [], 
                            "Subjects" TEXT [],
                            "PublishedYear" TEXT,
                            "Publisher" TEXT, 
                            "Pages" INTEGER, 
                            "Ratings" INTEGER, 
                            "Popularity" INTEGER, 
                            "Reviews" TEXT [], 
                            "OriginalLanguage" TEXT [],  
                            "Nationality" TEXT, 
                            "Translators" TEXT [],
                            "WorkTitles" TEXT [],
                            "Works" TEXT [],
                            "Languages" Text[],
                            "ISBN13" TEXT,
                            "EditionID" SERIAL,
                            PRIMARY KEY("EditionID") )
"""

q(qEditions)

In [6]:
qWorks = """
    CREATE TABLE works ( "WorkTitle" TEXT,
                        "Authors" TEXT [],
                        "Subjects" TEXT [],
                        "Description" TEXT,
                        "FirstPublishDate" TEXT,
                        "Editions" TEXT [],
                        "OtherTitles" TEXT [],
                        "TranslatedTitles" TEXT [],
                        "WorkID" SERIAL,
                        PRIMARY KEY ("WorkID") )
"""

q(qWorks)

In [8]:
class author():
    def __init__(self, dictionary_string):
        d = dictionary_string
        self.Name = d['name']
        self.Works = []
        self.Key = d['key'].split('/')[2]

In [14]:
author_data_dict = {'name':'Name',
                   'works':'Works',
                   'key':'Key'}

In [10]:
class edition():
    def __init__(self, dictionary):
        d = dictionary
        #Seperated Information
        self.Edition_Name = d['title']#
        self.Author_Name = d['authors']#
        self.Genre = d['genres']
        self.Subjects = d['subjects']
        self.Published_year = d['publish_date']
        self.Publisher = d['publishers']
        self.Pages = d['number_of_pages']
        self.Ratings = 0
        self.Popularity = 0
        self.Reviews = 0
        self.Original_Language = d['translated_from']
        self.Nationality = d['location']
        self.Translators = "{'-1'}"
        
        # Identifiers
#         self.identifiers = d['identifiers']#
#         self.isbn_10 = d['isbn_10']
#         self.isbn_13 = d['isbn_13']#-*
#         self.lccn = d['lccn']#
#         self.oclc = d['oclc']#

In [13]:
edition_data_dict = {'title':'Edition_Name',
                    'authors':'Author_Name',
                    'genres':'Genre',
                    'subjects':'Subjects',
                    'publish_date':'Published_year',
                    'publishers':'Publisher',
                    'number_of_pages':'Pages',
                    'ratings':'Ratings',
                    'popularity':'Popularity',
                    'reviews':'Reviews',
                    'translated_from':'Original_Language',
                    'location':'Nationality',
                    'translators':'Translators'}

In [11]:
class work():
    def __init__(self, dictionary):
        d = dictionary
        self.title = d['title']#
        self.authors = d['authors']#
        self.subjects = d['subjects']
        self.sub_loc = d['subject_places']
        self.sub_time = d['subject_times']
        self.sub_people = d['subject_people']
        self.orig_lang = d['original_languages']
        self.dewey = d['dewey_number']

In [12]:
def create_editions(input_string, work_type):
    try:
        if(type(input_string) != dict):
            input_string = eval(input_string)
        if(work_type == 'edition'):
            e = edition(input_string)
        elif(work_type == 'author'):
            e = author(input_string)
        else:
            e = work(input_string)
        return e
    except KeyError as e:
        e = e.__str__().replace('\'', '')
        input_string[e] = "{'-1'}"
        return create_editions(input_string, work_type)
    except NameError as e:
        input_string = input_string.replace('null', "{'-1'}")
        input_string = input_string.replace('true', 'True')
        input_string = input_string.replace('false', 'False')
        return create_editions(input_string, work_type)
    except Exception as e:
        print('Error in Create Editions')
        print(e)

In [13]:
def stringify(input_list):
    l = []
    
    for x in input_list:
        if type(x) == str:
            l.append('\''+x.replace("'", '').replace('"', '')+'\'')

        elif type(x) == list:
            try:
                base = 'ARRAY ['+','.join(process_items(x))+']'
            except TypeError as te:
                ser = process_items(x[0].values())
                base = 'ARRAY ['+','.join(ser)+']'
            if base == 'ARRAY []':
                l.append('ARRAY [-1]')
            else:
                l.append(base)

        else:
            l.append(str(x))
    return l

In [14]:
def process_items(input_list):
    return ['\''+str(x).replace("'", '').replace('"', '')+'\'' for x in input_list]

In [15]:
def process_keys(input_keys):
    return ['"'+str(x)+'"' for x in input_keys]

In [16]:
import time as t

def to_database(input_list):
    if(len(input_list) == 0):
        print('Empty List')
        return True
    input_obj = input_list[0]
    obj_type = type(input_obj).__name__
    obj_keys = process_keys(list(input_obj.__dict__.keys()))
    table = ''
    if(obj_type == 'author'):
        table = 'Authors'
    elif(obj_type == 'edition'):
        table = 'Editions'
    else:
        return 'Unsupported Type'
    
    print('Table = {}'.format(table))
    qBase = 'INSERT INTO "{}" ({}) VALUES '
    qFinal = qBase.format(table, ','.join(obj_keys))
    
    print('\tgenerating final query')
    start = t.time()
    for obj in input_list:
        try:
            obj_values = stringify(list(obj.__dict__.values()))
            if('' in obj_values):
                for x in obj_values:
                    if (x == ''):
                        x = "{'-1'}"
            qFinal = qFinal + '('+','.join(obj_values)+'),'
        except Exception as e:
            pass
    
    qFinal = qFinal[:-1] + ';'
    print('\tGeneration Time: {}'.format(t.time()-start))
    print('\tquerying')
    start = t.time()
    q(qFinal)
    print('\tQuery time (s): {}'.format(t.time()-start))
    
    return

In [17]:
import psutil
import pickle

In [18]:
last_position = pickle.load(open('last_pos.pkl', 'rb'))

In [23]:
counter = 0

authors = []
editions = []


with open(r'D:\New folder (3)\ol_dump_2020-01-31.txt\ol_dump_2020-01-31.txt', 'r', encoding='utf-8') as f:
#    f.seek(last_position)
    while(True):
        line = f.readline()
        if(line == ''):
            break
        
        line = line.split('\t')
        type_line = line[0].strip()
        
        if(type_line == '/type/author'):
            a = create_editions(line[4], 'author')
            authors.append(a)
            
        if(type_line == '/type/edition'):
            e = create_editions(line[4], 'edition')
            editions.append(e)
            
        if(type_line == '/type/work'):
            pass
#           to_database(create_editions(line[4], 'work'))
        counter += 1
        last_position = f.tell()
        
        if(counter%50000 == 0):
            last_position = f.tell()
            print('Exporting current results and recreating empty lists')
            to_database(authors)
            to_database(editions)
            del(authors)
            del(editions)
            authors = []
            editions = []
            print('Continuing the data input\n\n')

to_database(authors)
to_database(editions)
del(authors)
del(editions)
authors = []
editions = []

KeyboardInterrupt: 

In [ ]:
pickle.dumps(last_position, open('last_pos.pkl'))

In [25]:
query_t = """SELECT
   *
FROM
   pg_catalog.pg_tables
WHERE
   schemaname != 'pg_catalog'
AND schemaname != 'information_schema';"""


response = q(query_t, verbosity=True)

In [26]:
response

[('public', 'authors', 'betterreadsadmin', None, True, False, False, False),
 ('public', 'jointable', 'betterreadsadmin', None, False, False, False, False),
 ('public', 'editions', 'betterreadsadmin', None, True, False, False, False),
 ('public', 'works', 'betterreadsadmin', None, True, False, False, False)]

In [8]:
types

[str, list, list, list, str, str, int, int, int, int, list, str, list, int]

In [16]:
types[1]()

[]